In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

In [2]:
data = 'WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(data)

In [3]:
# Dropping ID because its not needed for modelling
df = df.drop(columns=["customerID"])

In [4]:
# Identify unique categorical values 
numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
for col in df.columns:
    if col not in numerical_features:
        print(col, df[col].unique())
        print("-"*50)

gender <StringArray>
['Female', 'Male']
Length: 2, dtype: str
--------------------------------------------------
SeniorCitizen [0 1]
--------------------------------------------------
Partner <StringArray>
['Yes', 'No']
Length: 2, dtype: str
--------------------------------------------------
Dependents <StringArray>
['No', 'Yes']
Length: 2, dtype: str
--------------------------------------------------
PhoneService <StringArray>
['No', 'Yes']
Length: 2, dtype: str
--------------------------------------------------
MultipleLines <StringArray>
['No phone service', 'No', 'Yes']
Length: 3, dtype: str
--------------------------------------------------
InternetService <StringArray>
['DSL', 'Fiber optic', 'No']
Length: 3, dtype: str
--------------------------------------------------
OnlineSecurity <StringArray>
['No', 'Yes', 'No internet service']
Length: 3, dtype: str
--------------------------------------------------
OnlineBackup <StringArray>
['Yes', 'No', 'No internet service']
Length: 3, 

In [5]:
# Since there exist whitespace values, we must filter them out
print(f"There are {len(df[df['TotalCharges'] == " "])} empty values \' \'")

There are 11 empty values ' '


In [6]:
df['TotalCharges'] = df['TotalCharges'].replace(" ", "0.0")
df['TotalCharges'] = df['TotalCharges'].astype(float)

In [7]:
# checking the distribution of the target column
print(df["Churn"].value_counts())

Churn
No     5174
Yes    1869
Name: count, dtype: int64


In [8]:
# Label encoding of target column
df["Churn"] = df["Churn"].replace({"Yes" : 1, "No": 0})
df["Churn"] = df["Churn"].astype(int)  # Add this line to make sure the column become integer and not object. FACED THIS PROBLEM
print(df["Churn"].value_counts())

Churn
0    5174
1    1869
Name: count, dtype: int64


# Split the data

In [9]:
# splitting the features and target
X = df.drop(columns=['Churn'])
y = df['Churn']

In [10]:
# split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
pd.Series(y_train).value_counts()

Churn
0    4138
1    1496
Name: count, dtype: int64

In [17]:
models = {
    "Logistic Regression Model" : LogisticRegression(random_state=42),
    "Decision Tree Model" : DecisionTreeClassifier(random_state=42),
    "Random Forest Model" : RandomForestClassifier(random_state=42),
    "XGBoost Model" : XGBClassifier(random_state=42)
}

In [26]:
lr = XGBClassifier(random_state=42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(score)

ValueError: could not convert string to float: 'Female'

# CONCLUSION WE NEED TO ENCODE IN ORDER TO HANDLE CLASS-IMBALANCE AND PERFORM ML ON THE DATA. WITHOUT THAT, IT'S IMPOSSIBLE!